> # CINE21 배우 정보 크롤링

## 라이브러리

In [1]:
import pymongo
from pprint import pprint

from bs4 import BeautifulSoup
import requests
import re

## MongoDB 연결

In [2]:
# 몽고 클라이언트
conn = pymongo.MongoClient()
# cine21 DB 연결
db = conn.cine21
# actor Collection 연결
collection = db.actor

- URL: http://www.cine21.com/rank/person/content
- Method: POST
- Form Data
  - section: actor
  - period_start: 2020-05
  - gender: all
  - page: 1

## 크롤링

### 참고: 특수한 정규 표현식

```html
- Greedy(.*) vs Non-Greedy(.*?)
- 연습: https://regexr.com
- <li><span class="tit">직업</span>배우</li>
```

In [ ]:
# 배우 정보 리스트
actors_info_list = list()

# Request 파라미터
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2019-11'
post_data['gender'] = 'all'

# 페이지 순회 1 ~ 20
for i in range(1, 21):
    
    print(f">> Page: {i} Start")
    post_data['page'] = i

    res = requests.post(cine21_url, data=post_data)

    soup = BeautifulSoup(res.content, 'html.parser')
    actors = soup.select('Li.people_li div.name')
    hits = soup.select('ul.num_info li strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li span.grade')

    url = 'http://www.cine21.com'

    for index, actor in enumerate(actors):

        # 배우 이름
        actor_nm = re.sub('\(\w*\)','', actor.text)
        # 배우 흥행지수
        actor_hit = int(hits[index].text.replace(',',''))
        # 배우 랭킹
        actor_ranking = rankings[index].text
        # 출연 영화 목록
        movie_titles = movies[index].select('li a span')
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)

        # 배우 상세 페이지 링크
        actor_link = url + actor.select_one("a").attrs['href']
        # 상세 페이지 요청
        response_actor = requests.get(actor_link)
        # 상세 페이지 파싱
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        # 배우 상세 정보
        actor_details = default_info.select('li')

        # 배우 상세 정보 딕셔너리
        actor_info_dict = dict()
        actor_info_dict['이름'] = actor_nm
        actor_info_dict['흥행지수'] = actor_hit
        actor_info_dict['랭킹'] = actor_ranking
        actor_info_dict['출연영화'] = movie_title_list
        for actor_item in actor_details:

            # 필드
            actor_item_field = actor_item.select_one('span.tit').text

            # 값
            actor_item_value = re.sub('<span.*?>.*?</span>','', str(actor_item))
            actor_item_value = re.sub('<.*?>','',actor_item_value)

            # 딕셔너리 추가
            actor_info_dict[actor_item_field] = actor_item_value

        # 리스트 추가
        actors_info_list.append(actor_info_dict)

## MongoDB에 데이터 적재

In [ ]:
collection.insert_many(actors_info_list)

## 적재된 Documents 수 확인

In [3]:
collection.count_documents({})

140

## Data Handling

### limit

In [21]:
doc = collection.find({}).limit(3)
for d in doc:
    pprint(d)

{'_id': ObjectId('5fa7ff30adc79825f5f61a04'),
 '다른이름': 'Byung-hun Lee;BH Lee',
 '랭킹': '1',
 '생년월일': '1970-07-12',
 '성별': '남',
 '신장/체중': '177cm, 72kg',
 '이름': '이병헌',
 '직업': '배우',
 '출연영화': ['백두산', '남산의 부장들', '그것만이 내 세상', '내부자들', '광해, 왕이 된 남자', '공동경비구역 JSA'],
 '취미': '모자수집, 여행',
 '특기': '태권도, 스노우보드, 수영, 팔씨름',
 '학교': '한양대학교 불어문학과',
 '홈페이지': '\nhttp://www.leebyunghun.kr/\n',
 '흥행지수': 66911}
{'_id': ObjectId('5fa7ff30adc79825f5f61a05'),
 '다른이름': '김성훈; 河正宇',
 '랭킹': '2',
 '생년월일': '1978-03-11',
 '성별': '남',
 '신장/체중': '184cm, 75kg',
 '이름': '하정우',
 '직업': '배우',
 '출연영화': ['백두산', '클로젯', '멋진 하루', '아가씨', '범죄와의 전쟁 : 나쁜놈들 전성시대', '암살'],
 '취미': '피아노, 검도, 수영',
 '학교': '중앙대학교 연극학 학사',
 '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n',
 '흥행지수': 45515}
{'_id': ObjectId('5fa7ff30adc79825f5f61a06'),
 '랭킹': '3',
 '생년월일': '1970-09-01',
 '성별': '남',
 '소속사': '예당엔터테인먼트',
 '신장/체중': '180cm, 75kg',
 '이름': '황정민',
 '직업': '배우',
 '출연영화': ['다만 악에서 구하소서', '신세계', '부당거래', '국제시장', '아수라'],
 '특기': '농구, 악기연주',
 '학교': '서울예술대학 연극과 졸업'

### update -  필드 명 변경

In [20]:
collection.update_many({}, {"$rename":{"다른 이름": "다른이름"}})

### sort

In [29]:
docs = collection.find({}).sort("생년월일", pymongo.DESCENDING).limit(2)
for doc in docs:
    pprint(d)

{'_id': ObjectId('5fa7ff30adc79825f5f61a8e'),
 '랭킹': '139',
 '성별': '여',
 '이름': '문승아',
 '직업': '배우',
 '출연영화': ['소리도 없이'],
 '흥행지수': 1202}
{'_id': ObjectId('5fa7ff30adc79825f5f61a8e'),
 '랭킹': '139',
 '성별': '여',
 '이름': '문승아',
 '직업': '배우',
 '출연영화': ['소리도 없이'],
 '흥행지수': 1202}


### exists

In [33]:
docs = collection.find({'특기':{"$exists": True}}).sort("흥행지수",pymongo.ASCENDING).limit(2)
for doc in docs:
    pprint(doc)

{'_id': ObjectId('5fa7ff30adc79825f5f61a75'),
 '다른이름': 'Jeon Do Youn',
 '랭킹': '114',
 '생년월일': '1973-02-11',
 '성별': '여',
 '소속사': '노아엔터테인먼트',
 '신장/체중': '165cm, 45kg',
 '이름': '전도연',
 '직업': '배우',
 '출연영화': ['무뢰한', '지푸라기라도 잡고 싶은 짐승들', '접속', '밀양', '멋진 하루', '해피엔드'],
 '취미': '음악듣기, 영화감상',
 '특기': '수상스키, 포켓볼',
 '학교': '서울예술대학 방송연예',
 '흥행지수': 1850}
{'_id': ObjectId('5fa7ff30adc79825f5f61a66'),
 '랭킹': '99',
 '생년월일': '1971-01-31',
 '성별': '여',
 '소속사': '에이스타즈',
 '신장/체중': '165cm, 48kg',
 '이름': '이영애',
 '직업': '배우',
 '출연영화': ['나를 찾아줘', '봄날은 간다', '친절한 금자씨', '공동경비구역 JSA'],
 '취미': '수영, 승마, 피아노',
 '특기': '노래',
 '학교': '한양대 독어독문  -중앙대학교대학원 연극영화',
 '흥행지수': 2448}


In [36]:
docs = collection.find({'생년월일':{"$exists": False}}, {"이름": 1, "_id": 0}).limit(10)
for doc in docs:
    pprint(doc)

{'이름': '이정재'}
{'이름': '염정아'}
{'이름': '김소진'}
{'이름': '조현정'}
{'이름': '정우성'}
{'이름': '김경덕'}
{'이름': '옥자연'}
{'이름': '김영은'}
{'이름': '이봉련'}
{'이름': '장동주'}


### 범위 쿼리

In [41]:
docs = collection.find({"흥행지수": {"$gte": 20000}}, {"_id":0,"이름":1,"흥행지수":1,"출연영화":1}).sort("흥행지수",pymongo.DESCENDING)
for doc in docs:
    pprint(doc)

{'이름': '이병헌',
 '출연영화': ['백두산', '남산의 부장들', '그것만이 내 세상', '내부자들', '광해, 왕이 된 남자', '공동경비구역 JSA'],
 '흥행지수': 66911}
{'이름': '하정우',
 '출연영화': ['백두산', '클로젯', '멋진 하루', '아가씨', '범죄와의 전쟁 : 나쁜놈들 전성시대', '암살'],
 '흥행지수': 45515}
{'이름': '황정민',
 '출연영화': ['다만 악에서 구하소서', '신세계', '부당거래', '국제시장', '아수라'],
 '흥행지수': 40079}
{'이름': '이정재',
 '출연영화': ['다만 악에서 구하소서', '신세계', '인천상륙작전', '사바하', '암살', '도둑들'],
 '흥행지수': 26719}
{'이름': '박정민',
 '출연영화': ['시동', '다만 악에서 구하소서', '사냥의 시간', '타짜: 원 아이드 잭', '동주', '들개'],
 '흥행지수': 26412}
{'이름': '전혜진', '출연영화': ['백두산', '뺑반', '불한당: 나쁜 놈들의 세상', '사도'], '흥행지수': 26197}
{'이름': '이성민',
 '출연영화': ['남산의 부장들', '미스터 주: 사라진 VIP', '부당거래', '뺑반', '밀양', '변호인'],
 '흥행지수': 22110}
{'이름': '유아인',
 '출연영화': ['＃살아있다', '소리도 없이', '버닝', '국가부도의 날', '사도', '우아한 거짓말'],
 '흥행지수': 21813}
{'이름': '곽도원',
 '출연영화': ['남산의 부장들', '강철비2: 정상회담', '국제수사', '무뢰한', '변호인', '아수라'],
 '흥행지수': 20093}


### 리스트 내 조회

In [46]:
docs = collection.find({'출연영화': "남산의 부장들"}, {"_id":0,"이름":1,"랭킹":1,"출연영화":1})
for doc in docs:
    pprint(doc)

{'랭킹': '1',
 '이름': '이병헌',
 '출연영화': ['백두산', '남산의 부장들', '그것만이 내 세상', '내부자들', '광해, 왕이 된 남자', '공동경비구역 JSA']}
{'랭킹': '7',
 '이름': '이성민',
 '출연영화': ['남산의 부장들', '미스터 주: 사라진 VIP', '부당거래', '뺑반', '밀양', '변호인']}
{'랭킹': '9',
 '이름': '곽도원',
 '출연영화': ['남산의 부장들', '강철비2: 정상회담', '국제수사', '무뢰한', '변호인', '아수라']}
{'랭킹': '15',
 '이름': '이희준',
 '출연영화': ['남산의 부장들', '오! 문희', '환상속의 그대', '미성년', '미쓰백', '최악의 하루']}
{'랭킹': '42', '이름': '김소진', '출연영화': ['남산의 부장들', '아이 캔 스피크', '미성년', '더 킹']}
{'랭킹': '122',
 '이름': '서현우',
 '출연영화': ['남산의 부장들', '나를 찾아줘', '테우리', '보희와 녹양', '국도극장', '너와 극장에서']}
{'랭킹': '138', '이름': '지현준', '출연영화': ['남산의 부장들']}
